In [1]:
using Pkg
Pkg.activate(".")
#Pkg.add(["Flux", "DLProteinFormats", "Onion", "RandomFeatureMaps", "StatsBase", "Plots"])

  Activating project at `c:\Users\User\Desktop\SoFo\code\SoFo-2025-Translation-Equivariant-Transformer`


In [2]:
using Flux, DLProteinFormats, RandomFeatureMaps, StatsBase, Plots

MethodError: MethodError: no method matching pipe_writer(::VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)})
The applicable method may be too new: running in world age 26711, while current world is 26750.

Closest candidates are:
  pipe_writer(::VSCodeServer.IJuliaCore.IJuliaStdio) (method too new to be called from this world context.)
   @ VSCodeServer c:\Users\User\.cursor\extensions\julialang.language-julia-1.149.2\scripts\packages\IJuliaCore\src\stdio.jl:16
  pipe_writer(!Matched::Base.ProcessChain)
   @ Base process.jl:37
  pipe_writer(!Matched::Pipe)
   @ Base stream.jl:780
  ...


In [3]:
includet("./RoPE.jl")
includet("./GQAttention.jl")

ERROR: LoadError: UndefVarError: `Flux` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at c:\Users\User\Desktop\SoFo\code\SoFo-2025-Translation-Equivariant-Transformer\RoPE.jl:25
in expression starting at c:\Users\User\Desktop\SoFo\code\SoFo-2025-Translation-Equivariant-Transformer\RoPE.jl:25
ERROR: UndefVarError: `Attention` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Stacktrace:
 [1] top-level scope
   @ c:\Users\User\Desktop\SoFo\code\SoFo-2025-Translation-Equivariant-Transformer\GQAttention.jl:3
in expression starting at c:\Users\User\Desktop\SoFo\code\SoFo-2025-Translation-Equivariant-Transformer\GQAttention.jl:3


In [4]:
dat = DLProteinFormats.load(PDBSimpleFlat500);

L = 30
train_inds = findall(dat.len .> L)
 
function random_batch(dat, L, B, filt_inds)
    locs = zeros(Float32, 3, L, B)
    inds = sample(filt_inds, B, replace=false)
    AAs = zeros(Int, L, B)
    for (i,ind) in enumerate(inds)
        l_range = rand(1:dat[ind].len - L + 1)
        locs[:, :, i] = dat[ind].locs[:, 1, l_range:l_range+L-1]
        AAs[:, i] = dat[ind].AAs[l_range:l_range+L-1]
    end
    return (;locs, AAs = Flux.onehotbatch(AAs, 1:20))
end

batch = random_batch(dat, L, 10, train_inds);
 
struct Toy0{L}
    layers::L
end
Flux.@layer Toy0
function Toy0()
    layers = (;
        AA_decoder = Dense(3 => 20, bias=false),
    )
    return Toy0(layers)
end
function (m::Toy0)(locs)
    l = m.layers
    aa_logits = l.AA_decoder(locs)
    return aa_logits
end
 
struct Toy1{L}
    layers::L
end
Flux.@layer Toy1
function Toy1(dim, depth)
    layers = (;
        loc_encoder = Dense(3 => dim, bias=false),
        transformers = [Onion.TransformerBlock(dim, 8, MultiDimRoPE(Int(dim/8), 3)) for _ in 1:depth],
        AA_decoder = Dense(dim => 20, bias=false),
    )
    return Toy1(layers)
end
function (m::Toy1)(locs)
    l = m.layers
    x = l.loc_encoder(locs)
    for transformerblock in l.transformers
        x = transformerblock(x, 0, nothing, locs)
        #locs = updatelocs(x, locs)
    end
    aa_logits = l.AA_decoder(x)
    return aa_logits
end
 
struct Toy2{L}
    layers::L
end
Flux.@layer Toy2
function Toy2(dim, depth)
    layers = (;
        loc_rff = RandomFourierFeatures(3 => 64, 0.1f0),
        loc_encoder = Dense(64 => dim, bias=false),
        transformers = [Onion.TransformerBlock(dim, 8) for _ in 1:depth],
        AA_decoder = Dense(dim => 20, bias=false),
    )
    return Toy2(layers)
end
function (m::Toy2)(locs)
    l = m.layers
    x = l.loc_encoder(l.loc_rff(locs))
    for layer in l.transformers
        x = layer(x, 0, nothing)
    end
    aa_logits = l.AA_decoder(x)
    return aa_logits
end
 

UndefVarError: UndefVarError: `DLProteinFormats` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
#model = Toy0()
model = Toy1(64, 4)
#model = Toy2(64, 4)
opt_state = Flux.setup(AdamW(eta = 0.001), model)
 
losses = Float32[]

UndefVarError: UndefVarError: `Toy1` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [6]:
model.layers.transformers[1]

UndefVarError: UndefVarError: `model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
for epoch in 1:20 # 1:100
    tot_loss = 0f0
    for i in 1:1_000 # 1:10_000
        batch = random_batch(dat, L, 10, train_inds);
        l, grad = Flux.withgradient(model) do m
            aalogits = m(batch.locs)
            Flux.logitcrossentropy(aalogits, batch.AAs)
        end
        Flux.update!(opt_state, model, grad[1])
        tot_loss += l
        if mod(i, 50) == 0
            println(epoch, " ", i, " ", tot_loss/50)
            push!(losses, tot_loss/50)
            tot_loss = 0f0
        end
        (mod(i, 500) == 0) && savefig(plot(losses), "losses_toy_MultiDimRoPE.pdf")
    end
end


UndefVarError: UndefVarError: `random_batch` not defined in `Main`
Suggestion: check for spelling errors or missing imports.